In [1]:
import torch
from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('/kaggle/input/minedataset/cleaned data - Sheet1.csv')

In [4]:
df['classes_id'].value_counts()

0    116
1     76
2     30
4     29
5     27
3     24
Name: classes_id, dtype: int64

In [5]:
df

,accident_code,accident_type,accident_classes,classes_id,reason
0,335,Dumpers,Dumpers,0,While dumpers were being guided by an official...
1,335,Dumpers,Dumpers,0,While a helper of a tipper was removing stop b...
2,335,Dumpers,Dumpers,0,While a loaded dumper having a capacity of 85 ...
3,335,Dumpers,Dumpers,0,While a loaded dumper having a capacity of 170...
4,335,Dumpers,Dumpers,0,While a 35T dumper was being reversed on coal ...
...,...,...,...,...,...
297,449,Other Non-Transportation Machinery,Other Heavy Earth Moving Machinery and non tra...,5,While a Powered Support was being raised in a ...
298,449,Other Non-Transportation Machinery,Other Heavy Earth Moving Machinery and non tra...,5,While a helper had approached in the close pro...
299,449,Other Non-Transportation Machinery,Other Heavy Earth Moving Machinery and non tra...,5,While a fan attendant approached near the driv...
300,449,Other Non-Transportation Machinery,Other Heavy Earth Moving Machinery and non tra...,5,"While a crew of five persons, engaged in measu..."


In [6]:
df=df[['reason','accident_classes']]

In [7]:
df

,reason,accident_classes
0,While dumpers were being guided by an official...,Dumpers
1,While a helper of a tipper was removing stop b...,Dumpers
2,While a loaded dumper having a capacity of 85 ...,Dumpers
3,While a loaded dumper having a capacity of 170...,Dumpers
4,While a 35T dumper was being reversed on coal ...,Dumpers
...,...,...
297,While a Powered Support was being raised in a ...,Other Heavy Earth Moving Machinery and non tra...
298,While a helper had approached in the close pro...,Other Heavy Earth Moving Machinery and non tra...
299,While a fan attendant approached near the driv...,Other Heavy Earth Moving Machinery and non tra...
300,"While a crew of five persons, engaged in measu...",Other Heavy Earth Moving Machinery and non tra...


In [8]:
possible_labels = df.accident_classes.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{'Dumpers': 0,
 'Fall of roof_sides_overhangs': 1,
 'Wheeled Trackless': 2,
 'Loading Machines': 3,
 'Fall of Person from Height/into Depth and on same level': 4,
 'Other Heavy Earth Moving Machinery and non transportation machinery': 5}

In [9]:
df['label'] = df.accident_classes.replace(label_dict)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [10]:
df.head()

,reason,accident_classes,label
0,While dumpers were being guided by an official...,Dumpers,0
1,While a helper of a tipper was removing stop b...,Dumpers,0
2,While a loaded dumper having a capacity of 85 ...,Dumpers,0
3,While a loaded dumper having a capacity of 170...,Dumpers,0
4,While a 35T dumper was being reversed on coal ...,Dumpers,0


In [11]:
# from sklearn.model_selection import train_test_split

# X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
#                                                   df.label.values, 
#                                                   test_size=0.15, 
#                                                   random_state=42,
#                                                   stratify=df.label.values
#                                                   )
from sklearn.model_selection import KFold

# Define the number of folds
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Split the data into folds
for train_index, val_index in kf.split(df):
    X_train, X_val = df.index.values[train_index], df.index.values[val_index]
    y_train, y_val = df.label.values[train_index], df.label.values[val_index]

In [12]:
df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [13]:
df.groupby(['accident_classes', 'label', 'data_type']).count()

reason
accident_classes                                   label data_type        
Dumpers                                            0     train          92
                                                         val            24
Fall of Person from Height/into Depth and on sa... 4     train          24
                                                         val             5
Fall of roof_sides_overhangs                       1     train          60
                                                         val            16
Loading Machines                                   3     train          20
                                                         val             4
Other Heavy Earth Moving Machinery and non tran... 5     train          21
                                                         val             6
Wheeled Trackless                                  2     train          25
                                                         val             5

In [14]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [15]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].reason.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].reason.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [16]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [17]:
len(dataset_train), len(dataset_val)

(242, 60)

In [18]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [19]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 3

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [20]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=3e-5, 
                  eps=1e-8)

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [21]:
epochs = 10

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [22]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [23]:
import numpy as np

In [24]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [25]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [26]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [27]:
import os

output_dir = 'data_volume'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)


In [28]:

from sklearn.metrics import f1_score

In [34]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'data_volume/finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    
   
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')
   
    

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/81 [00:00<?, ?it/s]


Epoch 1
Training loss: 0.01688428336099066
Validation loss: 0.6395443325745873
F1 Score (Weighted): 0.8449848484848486


Epoch 2:   0%|          | 0/81 [00:00<?, ?it/s]


Epoch 2
Training loss: 0.015353260768784417
Validation loss: 0.6395443325745873
F1 Score (Weighted): 0.8449848484848486


...


Epoch 8
Training loss: 0.01548584360761545
Validation loss: 0.6395443325745873
F1 Score (Weighted): 0.8449848484848486


Epoch 9:   0%|          | 0/81 [00:00<?, ?it/s]


Epoch 9
Training loss: 0.016197968711271698
Validation loss: 0.6395443325745873
F1 Score (Weighted): 0.8449848484848486


Epoch 10:   0%|          | 0/81 [00:00<?, ?it/s]


Epoch 10
Training loss: 0.015414537085841099
Validation loss: 0.6395443325745873
F1 Score (Weighted): 0.8449848484848486


In [35]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

model.load_state_dict(torch.load('data_volume/finetuned_BERT_epoch_1.model', map_location=torch.device('cpu')))

_, predictions, true_vals = evaluate(dataloader_validation)
accuracy_per_class(predictions, true_vals)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Class: Dumpers
Accuracy: 24/24

Class: Fall of roof_sides_overhangs
Accuracy: 15/16

Class: Wheeled Trackless
Accuracy: 4/5

Class: Loading Machines
Accuracy: 2/4

Class: Fall of Person from Height/into Depth and on same level
Accuracy: 3/5

Class: Other Heavy Earth Moving Machinery and non transportation machinery
Accuracy: 3/6

